In [1]:
import pickle
import os.path
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request

import pandas as pd
import numpy as np
import datetime
from gspread_dataframe import set_with_dataframe
import gspread

In [2]:
# If modifying these scopes, delete the file token.pickle.
SCOPES = ['https://www.googleapis.com/auth/spreadsheets',
    'https://www.googleapis.com/auth/drive']

# The ID and range of a sample spreadsheet.
SPREADSHEET_ID = '10yF9klJ-3ODLO1BbXUaT2E0ZSJMXmjz6TuNJL2vDdTc'


"""Shows basic usage of the Sheets API.
Prints values from a sample spreadsheet.
"""
creds = None
# The file token.pickle stores the user's access and refresh tokens, and is
# created automatically when the authorization flow completes for the first
# time.
if os.path.exists('token.pickle'):
    with open('token.pickle', 'rb') as token:
        creds = pickle.load(token)
# If there are no (valid) credentials available, let the user log in.
if not creds or not creds.valid:
    if creds and creds.expired and creds.refresh_token:
        creds.refresh(Request())
    else:
        flow = InstalledAppFlow.from_client_secrets_file(
            'credentials.json', SCOPES)
        creds = flow.run_local_server(port=0)
    # Save the credentials for the next run
    with open('token.pickle', 'wb') as token:
        pickle.dump(creds, token)

service = build('sheets', 'v4', credentials=creds)

# Call the Sheets API
sheet = service.spreadsheets()

In [3]:
ranges = ['transactions!A1:D', 'clients!A1:C', 'managers!A1:C', 'leads!A1:F'] 

# Download transactions sheet
result = sheet.values().get(spreadsheetId=SPREADSHEET_ID,
                            range=ranges[0]).execute()
values = result.get('values', [])
transactions_df = pd.DataFrame(data= values[1:], columns=values[0])
transactions_df['m_real_amount'] = transactions_df['m_real_amount'].astype(int)

# Download transactions clients
result = sheet.values().get(spreadsheetId=SPREADSHEET_ID,
                            range=ranges[1]).execute()
values = result.get('values', [])
clients_df = pd.DataFrame(data= values[1:], columns=values[0])

# Download transactions managers
result = sheet.values().get(spreadsheetId=SPREADSHEET_ID,
                            range=ranges[2]).execute()
values = result.get('values', [])
managers_df = pd.DataFrame(data= values[1:], columns=values[0])


# Download transactions leads
result = sheet.values().get(spreadsheetId=SPREADSHEET_ID,
                            range=ranges[3]).execute()
values = result.get('values', [])
leads_df = pd.DataFrame(data= values[1:], columns=values[0])

* В leads 824 строки с client_id = 00000000-0000-0000-0000-000000000000

* И еще 9 строк c4447d16-29bf-11eb-ac1f-c412f533dba1

* В leads есть 21 менеджер, которых нет в таблице managers. На них приходятся 566 заявки в leads

* Предположил, что в d_utm_source vk = vkontakte, insta = instagram, utm_source = ''. Поменял

* Предположил, что в d_utm_medium utm_medium = ''. Поменял

* Не ясно, что делать с метриками по продажам (продажи в течении недели после заявки), тк есть повторяющиеся клиенты в заявках и у разных менеджеров с разными датами. Для каждой такой записи +7 недель отсчитывается от данной даты.

* Много времени занимает операция по поиску мусорных заявок. Перебор таблицы клиентов не лучший вариант. Заметил, что мусорными являются клиенты с id = 00000000-0000-0000-0000-000000000000. Если это не случайность, а закономерность, то скрипт ускорится дважды.

In [4]:
managers_df.set_index(managers_df.columns[0], inplace=True)
unknown_managers = []
for i in leads_df.l_manager_id.unique():
    if i not in list(managers_df.index):
        unknown_managers.append(i)

In [5]:
data = leads_df.copy()
data['d_club'] = leads_df.apply(lambda x: managers_df['d_club'].loc[x.l_manager_id]
                                if x.l_manager_id not in unknown_managers else 'Unknown' , axis=1)
data['d_manager'] = leads_df.apply(lambda x: managers_df['d_manager'].loc[x.l_manager_id]
                                if x.l_manager_id not in unknown_managers else 'Unknown' , axis=1)
# data['created_at+l_client_id'] = leads_df.apply(lambda x: x['created_at'] + '--' + x['l_client_id'], axis=1)

data.d_utm_source.replace('insta', 'instagram', inplace=True)
data.d_utm_source.replace('vkontakte', 'vk', inplace=True)
data.d_utm_source.replace('utm_source', '', inplace=True)
data.d_utm_source.replace('ycard#!/tproduct/225696739-1498486363994', 'ycard', inplace=True)
data.d_utm_source.replace('', 'unknown', inplace=True)

data.d_utm_medium.replace('utm_medium', '', inplace=True)
data.d_utm_medium.replace('', 'unknown', inplace=True)

In [6]:
pivoted = data.pivot_table(values=['lead_id'], index=['d_utm_source', 'd_club', 'd_manager','l_client_id', 'created_at'],
                            aggfunc=lambda x: len(x.unique()))

In [7]:
%%time
# Создаем маску для уменьшения размера clients_df, чтобы быстрее искать "мусорных клиентов"
mask = [True if x in list(managers_df.index) else False for x in list(clients_df['l_manager_id'])]
clients_df_masked = clients_df[mask].copy()

# словарь с метриками 
metrics = {}
metrics['count_clients'] = []
metrics['count_sales_per_client'] = []
metrics['count_new_clients'] = []
metrics['count_loyal_clients'] = []
metrics['first_week_profit'] = []
metrics['first_week_profit_per_client'] = []
metrics['invalid_leads'] = []
metrics['count_leads_returned_clients'] = []
metrics['count_new_leads'] = []

# Идем перебором по всем заявкам в leads_df
utm_source_list = sorted(list(set([x[0] for x in list(pivoted.index)])))

for utm_source in utm_source_list:
    
    clubs_list = sorted(list(set([x[0] for x in list(pivoted.loc[utm_source].index)])))
    
    for club in clubs_list:
        
        mangers_list = sorted(list(set([x[0] for x in list(pivoted.loc[(utm_source, club)].index)])))
        
        for manager in mangers_list:
            p = pivoted.loc[(utm_source, club, manager)]

            cnt1 = 0
            cnt2 = 0
            cnt3 = 0
            cnt4 = 0
            cnt5 = 0
            cnt6 = 0
            sales_number = 0
            sales_amount = 0
            
            # Итерируемся по записям из сводной таблицы
            for i, source in p.iterrows():

                created_at = source.name[1]
                client_id = source.name[0]
                
                # количество мусорных заявок (на основании заявки не создан клиент)
                # очень затратная операция, надо оптимизировать
                if client_id not in list(clients_df_masked['client_id']):
                    cnt4 += 1
                    
                if '00000000-00' in client_id:
                    continue
                
                # Рассчитываем дату через неделю
                date_time_obj = datetime.datetime.strptime(created_at, '%Y-%m-%d %H:%M:%S')
                delta = datetime.timedelta(days=7)
                date_week = date_time_obj + delta
                date_week = date_week.strftime('%Y-%m-%d %H:%M:%S')
                
                # Рассчитываем дату через 3 месяца
                delta = datetime.timedelta(days=90)
                date_3months = date_time_obj - delta
                date_3months = date_3months.strftime('%Y-%m-%d %H:%M:%S')
                
                # Фильтруем записи по транзакциям в transactions_df
                first_week = transactions_df[(transactions_df['l_client_id'] == client_id) &
                                       (transactions_df['created_at'] < date_week) &
                                       (transactions_df['created_at'] >= created_at)]
                
                sales_first_week = len(first_week)
                
                sales_3months_before = len(transactions_df[(transactions_df['l_client_id'] == client_id) &
                                       (transactions_df['created_at'] < created_at) &
                                       (transactions_df['created_at'] >= date_3months)])
                
                sales_from_3month_before = len(transactions_df[(transactions_df['l_client_id'] == client_id) &
                                       (transactions_df['created_at'] < date_3months)])
                
                leads_anytime_before = len(data[(data['l_client_id'] == client_id) &
                                          (data['created_at'] < created_at)])
                
                # количество покупателей (кто купил в течение недели после заявки)
                # доход от покупок, сделанных в течение недели после получения заявки
                if sales_first_week > 0:
                    cnt1 += 1
                    sales_number += sales_first_week
                    sales_amount += first_week['m_real_amount'].sum()
                
                # количество новых покупателей (кто купил в течение недели после заявки, и не покупал раньше)
                if (sales_first_week > 0 and sales_from_3month_before == 0 
                    and sales_3months_before == 0):
                    cnt2 += 1
                
                # количество заявок с возвратами старых клиентов (не было покупок от
                # этого клиента три месяца, но до этого были)
                if sales_3months_before == 0 and sales_from_3month_before > 0:
                    cnt3 += 1
                    
                # количество новых заявок (не было заявок и покупок от этого клиента раньше)
                if (leads_anytime_before == 0 and sales_from_3month_before == 0 
                    and sales_3months_before == 0):
                    cnt5 += 1
                
                # количество заявок с возвратами новых клиентов (не было покупок от этого клиента раньше,
                # но были заявки)
                if (leads_anytime_before > 0 and sales_from_3month_before == 0 
                    and sales_3months_before == 0):
                    cnt6 += 1
                
            # количество покупок на одного покупателя за неделю после получения заявки
            if cnt1 > 0:
                metrics['count_sales_per_client'].append(round(sales_number/cnt1, 2))
            else:
                metrics['count_sales_per_client'].append(0)
            
            # количество покупателей (кто купил в течение недели после заявки)
            metrics['count_clients'].append(cnt1)
            
            # количество новых покупателей (кто купил в течение недели после заявки, и не покупал раньше)
            metrics['count_new_clients'].append(cnt2)
            
            # количество заявок с возвратами старых клиентов (не было покупок от 
            # этого клиента три месяца, но до этого были)
            metrics['count_loyal_clients'].append(cnt3)
            
            # доход от покупок, сделанных в течение недели после получения заявки
            metrics['first_week_profit'].append(sales_amount)
            
            # тот же доход на одного покупателя
            if cnt1 > 0:
                metrics['first_week_profit_per_client'].append(round(sales_amount/cnt1, 2))
            else:
                metrics['first_week_profit_per_client'].append(0)
            
            # количество мусорных заявок (на основании заявки не создан клиент)
            metrics['invalid_leads'].append(cnt4)
            
            # количество новых заявок (не было заявок и покупок от этого клиента раньше)
            metrics['count_new_leads'].append(cnt5)
            
            # количество заявок с возвратами новых клиентов (не было покупок от этого клиента раньше,
            # но были заявки)
            metrics['count_leads_returned_clients'].append(cnt6)

Wall time: 1min 2s


In [54]:
# Создаем финальную pivot table с метрикой "количество заявок"
pivoted_final = data.pivot_table(values=['lead_id'], index=['d_utm_source', 'd_club', 'd_manager'],
                            aggfunc=lambda x: len(x))
pivoted_final = pivoted_final.rename(columns={'lead_id': 'count_leads'})


# Переносим метрики из словаря в pivot table
pivoted_final['count_sales'+'\n'+'per_client'] = metrics['count_sales_per_client']
pivoted_final['invalid_leads'] = metrics['invalid_leads'] 
pivoted_final['count_new_leads'] = metrics['count_new_leads']
pivoted_final['count_leads'+'\n'+'returned_clients'] = metrics['count_leads_returned_clients']
pivoted_final['count_loyal_clients'] = metrics['count_loyal_clients']
pivoted_final['count_clients'] = metrics['count_clients']
pivoted_final['count_new_clients'] = metrics['count_new_clients']
pivoted_final['first_week_profit'] = metrics['first_week_profit'] 
pivoted_final['first_week_profit'+ '\n' + 'per_client'] = metrics['first_week_profit_per_client']

In [52]:
pivoted_final

count_leads  count_sales_per_client  \
d_utm_source d_club  d_manager                                          
google       Unknown Unknown                7                     0.0   
             club #1 manager #1            21                     1.0   
                     manager #7             7                     1.0   
             club #2 manager #10           14                     0.0   
                     manager #4             9                     1.0   
...                                       ...                     ...   
ycard        club #1 manager #7             3                     1.0   
             club #2 manager #4             2                     1.0   
                     manager #8             4                     1.0   
             club #3 manager #13            1                     0.0   
             club #4 manager #2             4                     0.0   

                                  invalid_leads  count_new_leads  \
d_utm_source d_club  d_manager                                     
google       Unknown Unknown                  7                1   
             club #1 manager #1               9               12   
                     manager #7               1                6   
             club #2 manager #10              1                6   
                     manager #4               1                6   
...                                         ...              ...   
ycard        club #1 manager #7               0                3   
             club #2 manager #4               1                1   
                     manager #8               0                4   
             club #3 manager #13              0                1   
             club #4 manager #2               1                3   

                                  count_leads_returned_clients  \
d_utm_source d_club  d_manager                                   
google       Unknown Unknown                                 0   
             club #1 manager #1                              0   
                     manager #7                              0   
             club #2 manager #10                             3   
                     manager #4                              1   
...                                                        ...   
ycard        club #1 manager #7                              0   
             club #2 manager #4                              1   
                     manager #8                              0   
             club #3 manager #13                             0   
             club #4 manager #2                              0   

                                  count_loyal_clients  count_clients  \
d_utm_source d_club  d_manager                                         
google       Unknown Unknown                        0              0   
             club #1 manager #1                     0              3   
                     manager #7                     0              1   
             club #2 manager #10                    2              0   
                     manager #4                     1              2   
...                                               ...            ...   
ycard        club #1 manager #7                     0              1   
             club #2 manager #4                     0              1   
                     manager #8                     0              1   
             club #3 manager #13                    0              0   
             club #4 manager #2                     0              0   

                                  count_new_clients  first_week_profit  \
d_utm_source d_club  d_manager                                           
google       Unknown Unknown                      0                  0   
             club #1 manager #1                   3              64811   
                     manager #7                   1               9212   
             club #2 manager #10 

In [91]:
# Проверяем доступ к Google Sheet API, но уже для 
# библиотеки gspread и gspread-dataframe

gc = gspread.authorize(creds)
sh = gc.open_by_key(SPREADSHEET_ID)

In [92]:
# Delet sheet 
sh.del_worksheet(sh.worksheet('pivot'))

# Add sheet
sh.add_worksheet(title='pivot', rows='20', cols='20' )

# Select sheet and download data 
worksheet = sh.worksheet('pivot')
set_with_dataframe(worksheet, pivoted_final, include_index=True, resize=True) 

In [85]:
# Получаем SheetId дял таблицы pivot
def get_sheetId(service, SpreadSheetId, sheet_name):
    
    sheet_metadata = service.spreadsheets().get(spreadsheetId=SpreadSheetId).execute()
    sheets = sheet_metadata.get('sheets', '')

    return [x['properties']['sheetId'] for x in sheets if x['properties']['title'] == sheet_name][0]

sheetId = get_sheetId(service, SPREADSHEET_ID, 'pivot')

In [86]:
# Функция для объединения строк

def merge_rows(start, length, level, SheetId):
    request =  {'mergeCells': {                    
            'mergeType': 'MERGE_COLUMNS',
            'range': {
                'startRowIndex': start,
                'endRowIndex': start + length ,
                'sheetId': SheetId,
                'startColumnIndex': level,
                'endColumnIndex': level + 1
                }
        }}
    return request

In [87]:
# Объединяем строки в таблице по индексам

requests = []

utm_source_list = sorted(list(set([x[0] for x in list(pivoted.index)])))

level1 = 0
start1 = 1

level2 = 1
start2 = 1
for utm_source in utm_source_list:
    
    length1 = len(pivoted_final.loc[utm_source])
    requests.append(merge_rows(start1, length1, level1, sheetId))
    start1 += length1
    
    clubs_list = sorted(list(set([x[0] for x in list(pivoted.loc[utm_source].index)])))
    
    for club in clubs_list:
        
        length2 = len(pivoted_final.loc[(utm_source, club)])
        requests.append(merge_rows(start2, length2, level2, sheetId))
        start2 += length2

In [88]:
# Автоматически подстроить размер клеток под размер контента
dim = {
    'autoResizeDimensions': {
        'dimensions': {
            'sheetId': sheetId, 
            'dimension': 'COLUMNS', 
            'startIndex': 0, 
            'endIndex': 13
        }
    }
}

requests.append(dim)

# Выровнять колонки dimensions
align_dimensions = {'repeatCell': {
    'cell': {
      'userEnteredFormat': {
        'horizontalAlignment': "CENTER",
        'verticalAlignment': "TOP",
      }
    },
    'range': {
      'sheetId': sheetId,
      'startRowIndex': 1,
      'endRowIndex': len(pivoted_final),
      'startColumnIndex': 0,
      'endColumnIndex': 2,
    },
    'fields': "userEnteredFormat"
}}

requests.append(align_dimensions)

# Выровнять заголовки
align_header = {'repeatCell': {
    'cell': {
      'userEnteredFormat': {
        'horizontalAlignment': "CENTER",
        'verticalAlignment': "TOP",
      }
    },
    'range': {
      'sheetId': sheetId,
      'startRowIndex': 0,
      'endRowIndex': 1,
      'startColumnIndex': 0,
      'endColumnIndex': 3 + len(pivoted_final.columns),
    },
    'fields': "userEnteredFormat"
}}

requests.append(align_header)

# Заморозить заголовки
froze_header = {
      "updateSheetProperties": {
        "properties": {
          "sheetId": sheetId,
          "gridProperties": {
            "frozenRowCount": 1
          }
        },
        "fields": "gridProperties.frozenRowCount"
      }}

requests.append(froze_header)

In [90]:
feedback = sh.batch_update({"requests": requests})
